# Evaluieren der Embedding Modelle
Extrahieren der Textchunks für die Kontextdatenbank


In [1]:
import os
import glob
import pymupdf4llm
import chromadb
import time
from semantic_text_splitter import MarkdownSplitter
from transformers import AutoTokenizer, AutoModel
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from typing import List
from math import ceil
import pandas as pd
from chromadb import PersistentClient



load_dotenv()

hf_api_key = os.getenv("HF_API_KEY")
import torch
from types import MethodType
from torch import Tensor
import torch.nn.functional as F



In [2]:
pdf_files = glob.glob(os.path.join("/home/m/PycharmProjects/ModelEval/data", "*.pdf"))

results = []

# Process each PDF file
for pdf_path in pdf_files:
    try:
        # Parse the PDF using pymupdf4llm
        parsed_content = pymupdf4llm.to_markdown(pdf_path)

        # Store the results
        results.append({
            "filename": os.path.basename(pdf_path),
            "content": parsed_content
        })
        print(f"Successfully processed: {pdf_path}")

    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

Successfully processed: /home/m/PycharmProjects/ModelEval/data/ubuntu-server-guide-2024-01-22.pdf
Successfully processed: /home/m/PycharmProjects/ModelEval/data/postgresql-17-A4.pdf


In [3]:
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-small", use_fast=True)

test_text = results[0]["content"]
tokens = tokenizer.encode(test_text, add_special_tokens=False)
total_tokens = len(tokens)

total_characters = len(test_text)

average_chars_per_512_tokens = (total_characters / total_tokens) * 512
print(f"Average characters per 512 tokens: {average_chars_per_512_tokens:.2f}")


Token indices sequence length is longer than the specified maximum sequence length for this model (392480 > 512). Running this sequence through the model will result in indexing errors


Average characters per 512 tokens: 1743.23


In [4]:
# Split each document into chunks
splitter = MarkdownSplitter(1700)
for result in results:
    result["chunks"] = splitter.chunks(result["content"])

# Flatten into lists of texts & ids
all_texts = []
all_ids   = []
for res in results:
    fname = res["filename"]
    for idx, chunk in enumerate(res["chunks"]):
        all_ids.append(f"{fname}__chunk_{idx}")
        all_texts.append(chunk)

print(f"Total chunks to index: {len(all_texts)}")

Total chunks to index: 6838


In [5]:
results[0]["chunks"]

['**Ubuntu Server** is a version of the Ubuntu operating system designed and engineered as a backbone for the internet.\n\nUbuntu Server brings economic and technical scalability to your datacentre, public or private. Whether you want to\ndeploy an OpenStack cloud, a Kubernetes cluster or a 50,000-node render farm, Ubuntu Server delivers the best value\nscale-out performance available.\n## **In this documentation**\n\n\nTutorials\n\nGet started - a hands-on introduction to Ubuntu Server for new users\n\n\nTutorials How-to guides\nGet started - a hands-on introduction to Ubuntu Server for new users Step-by-step guides covering key operations and comm\n\nExplanation Reference\nConcepts - discussion and clarification of key topics Technical information - package specifications, APIs, a\n\n\nReference\nTechnical information - package specifications, APIs, a',
 '## **Project and community**\n\nUbuntu Server is a member of the Ubuntu family. It’s an open source project that welcomes communit

In [4]:
class ManualHFEmbedder:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to("cuda")

    def __call__(self, input: List[str]) -> List[List[float]]:
        inputs = self.tokenizer(input, return_tensors="pt", padding=True, truncation=True).to("cuda")
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]
        return embeddings.cpu().tolist()


# 2. Define the four embedding functions
models = {
    "chromadb_default": embedding_functions.DefaultEmbeddingFunction(),  # ONNX MiniLM
    "e5_large": ManualHFEmbedder("intfloat/multilingual-e5-large"),
    "e5_base": ManualHFEmbedder("intfloat/multilingual-e5-base"),
    "e5_small": ManualHFEmbedder("intfloat/multilingual-e5-small"),
}


In [7]:
# 3. Initialize persistent client (once)
client = chromadb.PersistentClient(path="./data/chromadb")

results_times = {}
BATCH_SIZE = 512
MAX_DOCS = 5461

for name, ef in models.items():
    print(f"\n--- Benchmarking {name} ---")

    col_name = f"chunks_{name}"
    # Remove any previous run of this collection
    try:
        client.delete_collection(name=col_name)
    except:
        pass

    # Create (or recreate) the collection
    col = client.create_collection(
        name=col_name,
        embedding_function=ef
    )

    # Prepare your capped set of chunks
    texts = all_texts[:MAX_DOCS]
    ids    = all_ids[:MAX_DOCS]

    # Batch‐wise embedding + insert
    num_batches = ceil(len(texts) / BATCH_SIZE)
    t0 = time.time()
    for i in range(num_batches):
        start = i * BATCH_SIZE
        end   = start + BATCH_SIZE
        batch_texts = texts[start:end]
        batch_ids   = ids[start:end]
        col.add(documents=batch_texts, ids=batch_ids)
        if name != "chromadb_default":
            torch.cuda.synchronize()
    elapsed = time.time() - t0

    print(f"{name} embed+insert time: {elapsed:.2f}s")
    results_times[name] = elapsed

# 4. Summary
print("\n=== Embedding & Insertion Times ===")
print(f"{'Model':<20} {'Time (s)':>10}")
for name, t in results_times.items():
    print(f"{name:<20} {t:10.2f}")




--- Benchmarking chromadb_default ---
chromadb_default embed+insert time: 120.04s

--- Benchmarking e5_large ---
e5_large embed+insert time: 201.67s

--- Benchmarking e5_base ---
e5_base embed+insert time: 64.18s

--- Benchmarking e5_small ---
e5_small embed+insert time: 28.24s

=== Embedding & Insertion Times ===
Model                  Time (s)
chromadb_default         120.04
e5_large                 201.67
e5_base                   64.18
e5_small                  28.24


In [5]:
questions = [
    "Wie starte ich den PostgreSQL-Server unter Ubuntu?",
    "How do I start the PostgreSQL server on Ubuntu?",
    "Wie überprüfe ich die Speichernutzung von PostgreSQL mit Standard-Tools?",
    "How do I check PostgreSQL memory usage with standard tools?",
    "Wie stoppe ich den PostgreSQL-Server unter Ubuntu?",
    "How do I stop the PostgreSQL server on Ubuntu?",
    "Wie erstelle ich einen neuen Benutzer in PostgreSQL?",
    "How do I create a new user in PostgreSQL?",
    "Wie führe ich ein Backup einer PostgreSQL-Datenbank durch?",
    "How do I perform a backup of a PostgreSQL database?",
    "Wie installiere ich PostgreSQL auf Ubuntu über die Kommandozeile?",
    "How do I install PostgreSQL on Ubuntu via command line?",
    "Wie finde ich das Datenverzeichnis eines PostgreSQL-Servers?",
    "How do I find the data directory of a PostgreSQL server?",
    "Wie konfiguriere ich eine Master-Slave-Replikation in PostgreSQL?",
    "How do I configure a master-slave replication in PostgreSQL?",
    "Wie erstelle ich einen Foreign Key mit Cascading Delete in PostgreSQL?",
    "How do I create a foreign key with cascading delete in PostgreSQL?",
    "Wie plane ich regelmäßige Wartungsjobs mit PostgreSQL-internen Mitteln (z.B. autovacuum)?",
    "How do I schedule regular maintenance jobs with PostgreSQL internal tools (e.g., autovacuum)?",
    "Wie optimiere ich eine langsame PostgreSQL-Abfrage mit EXPLAIN ANALYZE?",
    "How do I optimize a slow PostgreSQL query with EXPLAIN ANALYZE?",
    "Wie konfiguriere ich PostgreSQL über Umgebungsvariablen oder Konfigurationsdateien dauerhaft?",
    "How do I permanently configure PostgreSQL using environment variables or configuration files?",
    "Wie erstelle ich eine materialisierte Sicht in PostgreSQL und aktualisiere sie regelmäßig?",
    "How do I create a materialized view in PostgreSQL and update it regularly?",
    "Wie authentifiziere ich PostgreSQL-Zugriffe über Zertifikate oder SSH-Tunnel?",
    "How do I authenticate PostgreSQL access via certificates or SSH tunnels?",
    "Wie verwende ich Transaktionen mit COMMIT und ROLLBACK in PostgreSQL?",
    "How do I use transactions with COMMIT and ROLLBACK in PostgreSQL?",
    "Wie erstelle ich eine PostgreSQL Datenbank?",
    "How do I create a PostgreSQL database?"
]

hard_questions = [
    # Advanced Questions
    "Wie implementiere ich eine partitionierte Tabelle in PostgreSQL mit Bereichspartitionierung für Zeitstempel?",
    "Wie konfiguriere ich ein RAID 10 System unter Ubuntu Server mit mdadm?",
    "Wie optimiere ich den PostgreSQL-Arbeitsspeicherverbrauch für einen Server mit 64GB RAM?",
    "Wie erstelle ich eine benutzerdefinierte Aggregatfunktion in PostgreSQL mit PL/pgSQL?",
    "Wie implementiere ich ein automatisches Failover-System für PostgreSQL mit repmgr unter Ubuntu?",
    "Wie konfiguriere ich SELinux-Richtlinien für eine sichere PostgreSQL-Installation?",
    "Wie erstelle ich einen benutzerdefinierten Index-Typ in PostgreSQL für Volltextsuche in deutschen Texten?",
    "Wie setze ich Kernel-Parameter für optimale PostgreSQL-Leistung unter hoher Last in Ubuntu?",

    # Specific Technical Questions
    "Wie führe ich ein VACUUM FULL auf eine PostgreSQL-Tabelle durch, ohne den Betrieb zu unterbrechen?",
    "Wie konfiguriere ich die pg_hba.conf Datei für SCRAM-SHA-256 Authentifizierung?",
    "Wie nutze ich systemd, um PostgreSQL bei Absturz automatisch neu zu starten?",
    "Wie erstelle ich eine PostgreSQL-Erweiterung mit C in Ubuntu?",
    "Wie implementiere ich Row-Level Security in PostgreSQL für mandantenfähige Anwendungen?",
    "Wie konfiguriere ich AppArmor-Profile für PostgreSQL in Ubuntu?",
    "Wie benutze ich die Ubuntu Live Migration für laufende virtuelle Maschinen mit PostgreSQL?",
    "Wie löse ich das Problem 'too many connections' in PostgreSQL ohne Neustart des Servers?",

    # Combination/Scenario Questions
    "Wie konfiguriere ich einen Ubuntu-Server als PostgreSQL-Datenbank mit automatischem Backup auf S3?",
    "Wie migriere ich eine große PostgreSQL-Datenbank von Ubuntu 20.04 auf 22.04 mit minimaler Ausfallzeit?",
    "Wie überwache ich PostgreSQL-Performance-Metriken in Ubuntu mit Prometheus und Grafana?",
    "Wie richte ich eine hochverfügbare PostgreSQL-Umgebung mit Patroni und etcd in Ubuntu ein?",
    "Wie implementiere ich eine sichere PostgreSQL-Installation gemäß den DSGVO-Anforderungen unter Ubuntu?",
    "Wie optimiere ich sowohl das Ubuntu-Dateisystem als auch PostgreSQL für SSD-Speicher?",
    "Wie konfiguriere ich TLS/SSL für die PostgreSQL-Verbindungen auf einem Ubuntu-Server?",
    "Wie implementiere ich logische Replikation zwischen verschiedenen PostgreSQL-Versionen in Ubuntu?",
]

In [6]:
# 1) Configurable parameters
PERSIST_PATH = "project_root/data/chromadb"
top_k       = 1     # <-- change this if you want more than 1 result per query

# 2) Connect to your persistent DB and load collections
client = chromadb.PersistentClient(path="project_root/data/chromadb")
cols = {
    "default": client.get_collection("chunks_chromadb_default", embedding_function=models["chromadb_default"]),
    "small":   client.get_collection("chunks_e5_small", embedding_function=models["e5_small"]),
    "base":    client.get_collection("chunks_e5_base", embedding_function=models["e5_base"]),
    "large":   client.get_collection("chunks_e5_large", embedding_function=models["e5_large"]),
}


# 3) Prepare the results dict
results = {
    "question":       [],
    "time_default":   [],
    "time_small":     [],
    "time_base":      [],
    "time_large":     [],
    "texts_default":  [],
    "texts_small":    [],
    "texts_base":     [],
    "texts_large":    [],
}

# 4) For each question, query all four collections
for q in questions:
    results["question"].append(q)
    # default
    t0 = time.perf_counter()
    qr = cols["default"].query(query_texts=[q], n_results=top_k)
    t1 = time.perf_counter()
    results["time_default"].append(t1 - t0)
    results["texts_default"].append(qr["documents"][0])
    # small
    t0 = time.perf_counter()
    qr = cols["small"].query(query_texts=[q], n_results=top_k)
    t1 = time.perf_counter()
    results["time_small"].append(t1 - t0)
    results["texts_small"].append(qr["documents"][0])
    # base
    t0 = time.perf_counter()
    qr = cols["base"].query(query_texts=[q], n_results=top_k)
    t1 = time.perf_counter()
    results["time_base"].append(t1 - t0)
    results["texts_base"].append(qr["documents"][0])
    # large
    t0 = time.perf_counter()
    qr = cols["large"].query(query_texts=[q], n_results=top_k)
    t1 = time.perf_counter()
    results["time_large"].append(t1 - t0)
    results["texts_large"].append(qr["documents"][0])

# 5) Build DataFrame and export
df = pd.DataFrame.from_dict(results)
df.to_excel("./data/retrieval_results.xlsx", index=False)
print("Wrote retrieval_results.xlsx")


Wrote retrieval_results.xlsx
